<a href="https://colab.research.google.com/github/nalderto/POL300-Public/blob/master/modules/module-3-exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 3: Obtaining Data with Web Scraping

For this activity, you are going to start doing some basic web scraping using Python.  At the end of this notebook, there is a single exercise that covers the content you just read.

You will be graded on your code's ability to complete the task described.  All the exercises have test cases to check the functionality of your code.  Make sure the test cases print `OK`.  This means that your code is working fine.  If it doesn't print `OK`, it will tell you which test case is failing.  

To submit this Jupyter notebook to Gradescope, reference the instructions in the module 1 activity or the "Getting Started with Jupyter Hub" guide on Brightspace.

If you are having trouble, don't get discouraged.  Read through the section again.  If you are still having trouble, try to Google around.  There are **TONS** of good online Python resources.  If Google is not being helpful, reach out via the Brightspace Discussion Board.  

## Introduction to Beautiful Soup

When conducting textual analysis on website content, it becomes necessary to quickly import the appropriate data into Python. Luckily, Python developers have created hundreds of modules that facilitate this process. For now, we are going to focus on a package known as *Beautiful Soup*. It allows us to quickly pull data out of the HTML pages that create websites.

## Importing Modules

```python
from bs4 import BeautifulSoup
import requests
```

These line must be included at the beginning of your Python file. It basically tells Python that you want to use Beautiful Soup and Requests with this file. The Requests library is what is going to allow us to actually retrieve web pages from the internet. Beautiful Soup only does the processing once we have the appropriate files.

## Obtaining the Web Page

The first step of web scraping is downloading the target web page. To achieve this, we are going to be using the Requests module that we imported earlier. For this demonstration, we are going to extract the campaign contribution data for each of the Indiana House of Representative seats from Ballotpedia \([here is District 26 for example](https://ballotpedia.org/Indiana_House_of_Representatives_District_26)\).

```python
page = requests.get("https://ballotpedia.org/Indiana_House_of_Representatives_District_26")

if (page.status_code != 200):
    print("Error: Not able to obtain web page!")
```

This line of code used the `get()` function from the `requests` module to download the HTML data from the State of the Union web page. The `if` statement is checking the status code from this request. The `200` code means that we successfully downloaded the web page. If it is something else, we print an error message.

If you successfully downloaded the web page, you should be able to run `print(page.content)` to see all of the data from the HTML. There is a lot of extra stuff in this web page that we don't want to analyze, so we are going to use Beautiful Soup to only keep the speech portion of the document.

## Finding the Portion of the Web page for Extraction

In order to extract content from a web page, we first need to narrow the scope of the content we want to extract. We will need to tell Beautiful Soup specifically what content we want to extract.

### Step 0: Basic HTML Knowledge

HTML is a language that makes web pages.  It is comprised of HTML "tags."  Common tags include `<h1>` for large headers, `<p>` for paragraphs, `<tr>` for table rows, and `<div>` for dividing the document into sections.  Each of these tags can optionally have multiple "classes" or a single "id" associated with it.  These will be listed within the tag as so:

```markup
<h1 class="big" id="banner">
Super cool banner!
</h1>
<p class="highlight bold">More text!</p>
```

Notice how tags have ending tags like `</p>` to signify the end of the section.  Tags can also nest within each other.  Multiple classes will just be listed with a space between them.

This is essentially all you need to know about HTML to move forward.  There are lots of other tags that we didn't cover, but the purpose of these tags is irrelevant. We are going to be using these tags to identify unique tags that contain the portion of text that we want to extract. We will be using Google Chrome's DevTools in order to find the portion of web page we want to extract.   

### Step 1: Right click on the part of the text you want to extract and click the "Inspect" option

![right-click](https://user-images.githubusercontent.com/25762130/89661713-072dd580-d8a1-11ea-9298-9f1798a69b2a.png)

### Step 2: Hover over the various HTML tags on the right part of your screen until you find one that highlights all the portion of text you want to extract.  

In the best-case scenario, you will find an HTML tag that only encompasses the item you want to web scrape, although this is not always possible. So, you are looking for an HTML tag that contains the minimal amount of extra stuff beyond what you want to web scrape.  Alternatively, you might find that HTML tags repeat over all the sections you are interested in.

![hover](https://user-images.githubusercontent.com/25762130/89661620-e82f4380-d8a0-11ea-8a97-7691f7c5eb9a.png)

### Step 3: Determine the HTML class, id, or tag that contains you text. 

With the campaign contributions table on Ballotpedia, we can see that the table is contained by the `table` tag with the classes `cfttable`and `bptable`.  **It is important that your chosen class, ID, or tag is unique, otherwise you might extract data that you don't intend to obtain.**  In this case, while `tbody`contains the entire table we want, we need to specify the `<table class="cftable bptable">`, because there exists another `<tbody>` in the HTML document.

## Using Beautiful Soup

Essentially all that Beautiful Soup does is parse a web page and extract the data that we specify via HTML classes, IDs, or tags.  The actual code for this portion of web scraping is going to vary widely based upon the web page that we are scraping.  If we are lucky, the website will have a unique ID for the specific piece of data we are attempting to extract.  

We are fairly lucky with this Politico campaign contributions data, as it is contained in the `<table class="cftable bptable">`.  We are going to use two lines of code to extract this table from the Politico web page.  

```python
# page variable is the variable we recieved from the requests module
soup = BeautifulSoup(page.content, 'lxml')
table = soup.find("table", class_="cftable bptable")
```

The line `BeautifulSoup(page.content, 'lxml')` creates an instance of the Beautiful Soup module.  The first argument is a string of the website's HTML code.  In the code above, the `page` variable is what we received from `requests.get(url)`.  We use `.content` to get the HTML code as a string.  The second argument is the HTML parser.  We are going to use `lxml`, since it is the fastest available parser.  

The second line,`soup.find("table", class_="cftable bptable")`, extracts the campaign contributions table from the Politico web page.  The `find` function will extract the **first** instance that it finds in the web page.  If we want to find all instances, we could use `findAll()`, and it will return a list of all the instances of the specified identifier.

The first argument in `find()` represents the HTML tag that contains that content we are interested in \(like `<div>`,`<p>`, or `<table>`\).  We then specify the classes or ID by passing in `class_="class goes here"`or `id="id"` as additional arguments (notice the underscore after the `class_` keyword argument).

Now, if we were to `print(table)`, you should see that it contains that table data from the web page.  In the next section, we are going to cover how to clean that text up, so it is nicely formatted and easily accessible.

## Excercise

For this exercise, you are going to incorporate what you learned in this module to obtain the campaign contributions table for the [Indiana House of Representatives District 1 on Ballotpedia](https://ballotpedia.org/Indiana_House_of_Representatives_District_1).

1. Use the requests module to request the web page for Indiana House of Representatives District 1 on Ballotpedia.  Set it to the variable `page`.  You can pretty much do what we did above, except change the URL to the Indiana House of Representative District 1.

2. Next, use Beautiful Soup to to parse the web page and extract the campaign contributions table HTML.  This process is going to be very similar to the example above, but we would still recommend that your use the Chrome DevTools and look through the web page's HTML on your own.

3. Add a `print` statement that prints the table found by Beautiful Soup.  This is so you can confirm that you collected the right table.

4. Once you have extracted the data using Beautiful Soup, use a `return` statement to return table.


Write your code in the `get_table` function below.  

**Hint: Most of the code is already written for you in examples above.  You just need to change one thing and combine the various statements together.**

In [ ]:
from bs4 import BeautifulSoup
import requests

def get_table():
    # Write your code here

    
    
    
    
    
    
    
    
    
    
# IGNORE BELOW
# Test Cases
import unittest
import pandas as pd

  
class TestCases(unittest.TestCase): 
    # Returns True or False.  
    def test(self):
        # Pandas
        table = str(get_table())
        
        df = pd.read_html(table, header=1, index_col=False)[0]
        
        amount= ['$19,285', '$36,150', '$50,650', '$64,846', '$44,314', '$49,204', '$64,756', '$164,829', '$24,143', '$17,425']
        candidates = [2, 2, 1, 1, 2, 3, 3, 3, 1, 1]
        average = ['$9,643', '$18,075', '$50,650', '$64,846', '$22,157', '$16,401', '$21,585', '$54,943', '$24,143', '$17,425']
        for i in range(0, 10):
            self.assertTrue(df['Year'][i] == str(2018 - (i * 2)))
            self.assertTrue(df['Amount'][i] == amount[i])
            self.assertTrue(df['Candidates'][i] == candidates[i])
            self.assertTrue(df['Average'][i] == average[i])
        
if __name__ == '__main__': 
    unittest.main(argv=['first-arg-is-ignored'], exit=False)